In [1]:
import torch
import torch.nn as nn

In [8]:
class SELayer(nn.Module):
    def __init__(self, channnel, reduction=16) -> None:
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channnel, channnel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channnel // reduction, channnel, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [9]:
class SE_block(nn.Module):
    def __init__(self, channnel, reduction=16) -> None:
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        self.l1 = nn.Linear(channnel, channnel // reduction, bias=False)
        self.act = nn.ReLU(inplace=True)
        self.l2 = nn.Linear(channnel // reduction, channnel, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x)
        print(f'avg_pool shape : {y.shape}')
        y = y.view(b, c)
        print(f'view shape : {y.shape}')
        y = self.act(self.l1(y))
        print(f'l1 shape : {y.shape}')
        y = self.l2(y)
        print(f'l2 shape : {y.shape}')
        y = self.sigmoid(y)
        y = y.view(b, c, 1, 1)
        print(f'view shape : {y.shape}')
        return x * y.expand_as(x)

In [10]:
x = torch.randn((2, 10, 4, 4))
model = SE_block(10, 2)
y = model(x)
print(f'y shape : {y.shape}')

avg_pool shape : torch.Size([2, 10, 1, 1])
view shape : torch.Size([2, 10])
l1 shape : torch.Size([2, 5])
l2 shape : torch.Size([2, 10])
view shape : torch.Size([2, 10, 1, 1])
y shape : torch.Size([2, 10, 4, 4])
